# Use previously generated models

In [3]:
import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle
import gensim

from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import LsiModel

from gensim.models.ldamulticore import LdaMulticore
from gensim.models.ldamodel import LdaModel

In [3]:
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus('spacy_trigram_bow_corpus_all.mm') # No POS preprocessing

# load the finished dictionary from disk
trigram_dictionary = Dictionary.load('spacy_trigram_dict_all.dict') # No POS preprocessing

# load the finished bag-of-words corpus from disk
trigram_bow_corpus_POS = MmCorpus('spacy_trigram_bow_corpus_all_POS.mm') # With POS preprocessing

# load the finished dictionary from disk
trigram_dictionary_POS = Dictionary.load('spacy_trigram_dict_all_POS.dict') # With POS preprocessing

In [54]:
# %%time
# # Create LDA model

# with warnings.catch_warnings():
#     warnings.simplefilter('ignore')

#     # workers => sets the parallelism, and should be
#     # set to your number of physical cores minus one
#     lda = LdaModel(trigram_bow_corpus_POS, 
#                                  id2word=trigram_dictionary_POS, 
#                                  num_topics=25)
    
#     lda.save('spacy_lda_model_all_POS')

CPU times: user 1min 34s, sys: 663 ms, total: 1min 35s
Wall time: 1min 35s


In [4]:
# load the finished LDA model from disk
lda = LdaModel.load('spacy_lda_model_all_POS')

In [5]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print(u'{:20} {}'.format(u'term', u'frequency') + u'')

    for term, frequency in lda.show_topic(topic_number, topn=40):
        print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))

In [6]:
for i in range(25): # Same number as the types of jobs we scraped initially
    print("\n\nTopic %s" % str(i))
    explore_topic(topic_number=i)



Topic 0
term                 frequency
release              0.010
platform             0.009
role                 0.007
world                0.007
’s                   0.007
manager              0.007
people               0.007
part                 0.006
feature              0.006
organization         0.006
leadership           0.005
time                 0.005
quality              0.005
success              0.005
delivery             0.005
workday              0.005
cloud                0.005
agile                0.005
architecture         0.005
developer            0.005
enterprise           0.005
way                  0.004
veteran_status_disability 0.004
future               0.004
oracle               0.004
industry             0.004
senior               0.004
leader               0.004
status               0.004
complex              0.003
area                 0.003
level                0.003
expertise            0.003
splunk               0.003
orientation_age_marital_status 0.003

In [7]:
# Manually labelled; not accurate for LDA yet
topic_names = {0: u'sales_marketing_1',
               1: u'technical_sales',
               2: u'cloud_infrastructure',
               3: u'cloud_analytics',
               4: u'test_and_qa',
               5: u'technical_pm',
               6: u'dba',
               7: u'ux_user_research',
               8: u'data_science',
               9: u'security_testing',
               10: u'product_owner',
               11: u'devops',
               12: u'qa',
               13: u'pm',
               14: u'cloud_network_analyst',
               15: u'pm_general',
               16: u'engineering_general',
               17: u'network_engineering',
               18: u'data_science_research',
               19: u'analytics',
               20: u'general_cloud_eng',
               21: u'pm_data_science',
               22: u'analytics',
               23: u'performance_analyst',
               24: u'general_implementation'}

In [59]:
topic_names_filepath = 'topic_names.pkl'

with open(topic_names_filepath, 'wb') as f:
    pickle.dump(topic_names, f)

## Visualize Topics

In [8]:
LDAvis_data_filepath = 'ldavis_prepared'

In [12]:
%%time

LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus_POS,
                                          trigram_dictionary_POS)

with open(LDAvis_data_filepath, 'wb') as f:
    pickle.dump(LDAvis_prepared, f)

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


CPU times: user 1min 15s, sys: 289 ms, total: 1min 15s
Wall time: 1min 19s


In [14]:
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

In [15]:
pyLDAvis.display(LDAvis_prepared)

In [16]:
from gensim.models import Word2Vec
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

trigram_sentences = LineSentence('spacy_trigram_sentences_PARSED.txt')
word2vec_filepath = 'word2vec_model_all'

In [26]:
%%time

# initiate the model and perform the first epoch of training
skill2vec = Word2Vec(trigram_sentences, size=100, window=5,
                    min_count=20, sg=1, workers=3, iter=10)

skill2vec.save(word2vec_filepath)
print("Done generating model.")

# perform 10 epochs of training
skill2vec.train(trigram_sentences,total_examples=skill2vec.corpus_count, epochs=skill2vec.iter)
skill2vec.save(word2vec_filepath)
print("Finished epoch %s" % str(i))

In [4]:
# load the finished model from disk
skill2vec = Word2Vec.load(word2vec_filepath)
skill2vec.init_sims()
print(u'{} training epochs so far.'.format(skill2vec.train_count))

2 training epochs so far.


/home/goodgame/anaconda3/lib/python3.6/site-packages/gensim/utils.py:491: DeprecationWarning: Call to deprecated `cum_table` (Attribute will be removed in 4.0.0, use self.vocabulary.cum_table instead).
  setattr(self, attrib, None)
